In [1]:
from cameo import models
from cameo import load_model
import pandas as pd
import numpy as np
from cobra import Model, Reaction, Metabolite
import cobra

/home/matinnu/anaconda3/envs/lab_rotation_3/lib/python3.7/site-packages/sympy/__init__.py:676: SymPyDeprecationWarning: 

importing sympy.core.singleton with 'from sympy import *' has been
deprecated since SymPy 1.6. Use import sympy.core.singleton instead.
See https://github.com/sympy/sympy/issues/18245 for more info.

  deprecated_since_version="1.6").warn()


In [2]:
def evaluate_knockout(model, mutant, ko_list):
    bound_knockout = []
    for i in ko_list:
        i_gene = str(model.genes[i])
        i_uniprot = str(model.genes[i].annotation['uniprot'][0])
        i_n_reactions = len(model.genes[i].reactions)
        i_reactions = []
        i_bounds_model = []
        i_bounds_mutant = []
        for x in model.genes[i].reactions:
            x_reaction = [str(x.id), str(x.reaction)]
            x_bounds = x.bounds
            i_reactions.append(x_reaction)
            i_bounds_model.append(x_bounds)
        for y in mutant.genes[i].reactions:
            y_bounds = y.bounds
            i_bounds_mutant.append(y_bounds)
        data = [i_gene, i_uniprot, i_n_reactions, i_reactions, i_bounds_model, i_bounds_mutant]
        bound_knockout.append(data)

    for i in bound_knockout:
        print('gene: '+i[0], '| uniprot: '+i[1])
        print('number of reactions involved: '+str(i[2]))
        #for count, x in enumerate(i[3]):
        #    print('* '+x[0]+' :', x[1])
        #    print('bounds normal: '+str(i[4][count]))
        #    print('bounds mutant: '+str(i[5][count]))
        print('------------------------------------------------------------')
    
    gene_reaction_pair = []
    for i in bound_knockout:
        gene_id = i[0]
        uniprot_id = i[1]
        for count, x in enumerate(i[3]):
            reaction_id = x[0]
            reaction = x[1]
            b_normal = i[4][count]
            b_mutant = i[5][count]
            if b_normal == b_mutant:
                ko_result = False
            else:
                ko_result = True
            pair = [uniprot_id, gene_id, reaction_id, reaction, b_normal, b_mutant, ko_result]
            gene_reaction_pair.append(pair)
    df = pd.DataFrame(data = gene_reaction_pair ,columns=["Entry", "id_BIGG", "reaction_id", "reaction", "normal_bound", "mutant_bound", "ko_result"])
    return df

In [3]:
model = load_model('iML1515')
model.solver = 'gurobi'
model

Using license file /home/matinnu/gurobi.lic
Academic license - for non-commercial use only


Name,iML1515
Memory address,0x07f26c890e5d0
Number of metabolites,1877
Number of reactions,2712
Number of groups,0
Objective expression,0.0 + 1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


# Xylitol strain knockout model

In [4]:
# copy model to create mutant
mutant_xylitol = model.copy()

# create list of genes to KO
ko_list = pd.read_csv('data/df_xylitol_knockouts.csv', index_col=0).dropna(subset = ['id_bigg'])
ko_list.iML1515_idx = ko_list.iML1515_idx.astype(int)
#ko_list

Read LP format model from file /tmp/tmp0nlvvhxl.lp
Reading time = 0.04 seconds
: 1877 rows, 5424 columns, 21150 nonzeros
Read LP format model from file /tmp/tmplf8l1_8z.lp
Reading time = 0.04 seconds
: 1877 rows, 5424 columns, 21150 nonzeros


In [5]:
for i in ko_list.iML1515_idx:
    mutant_xylitol.genes[i].knock_out()

xylitol_gene_ko = evaluate_knockout(model, mutant_xylitol, ko_list.iML1515_idx)
xylitol_gene_ko.to_csv('result/iML1515_xylitol_strain_ko_result.csv')
xylitol_gene_ko

gene: b0904 | uniprot: P0AC23
number of reactions involved: 2
------------------------------------------------------------
gene: b0903 | uniprot: P09373
number of reactions involved: 2
------------------------------------------------------------
gene: b1380 | uniprot: P52643
number of reactions involved: 1
------------------------------------------------------------
gene: b1241 | uniprot: P0A9Q7
number of reactions involved: 3
------------------------------------------------------------
gene: b3564 | uniprot: P09099
number of reactions involved: 2
------------------------------------------------------------
gene: b4154 | uniprot: P00363
number of reactions involved: 2
------------------------------------------------------------
gene: b0431 | uniprot: P0ABI8
number of reactions involved: 1
------------------------------------------------------------
gene: b0979 | uniprot: P26458
number of reactions involved: 2
------------------------------------------------------------


,Entry,id_BIGG,reaction_id,reaction,normal_bound,mutant_bound,ko_result
0,P0AC23,b0904,FORt2pp,for_p + h_p --> for_c + h_c,"(0.0, 1000.0)","(0.0, 1000.0)",False
1,P0AC23,b0904,FORtppi,for_c --> for_p,"(0.0, 1000.0)","(0.0, 1000.0)",False
2,P09373,b0903,PFL,coa_c + pyr_c --> accoa_c + for_c,"(0.0, 1000.0)","(0.0, 1000.0)",False
3,P09373,b0903,OBTFL,2obut_c + coa_c --> for_c + ppcoa_c,"(0.0, 1000.0)","(0.0, 1000.0)",False
4,P52643,b1380,LDH_D,lac__D_c + nad_c <=> h_c + nadh_c + pyr_c,"(-1000.0, 1000.0)","(0, 0)",True
5,P0A9Q7,b1241,ALCD19,glyald_c + h_c + nadh_c <=> glyc_c + nad_c,"(-1000.0, 1000.0)","(-1000.0, 1000.0)",False
6,P0A9Q7,b1241,ALCD2x,etoh_c + nad_c <=> acald_c + h_c + nadh_c,"(-1000.0, 1000.0)","(-1000.0, 1000.0)",False
7,P0A9Q7,b1241,ACALD,acald_c + coa_c + nad_c <=> accoa_c + h_c + na...,"(-1000.0, 1000.0)","(-1000.0, 1000.0)",False
8,P09099,b3564,DXYLK,atp_c + dxyl_c --> adp_c + dxyl5p_c + h_c,"(0.0, 1000.0)","(0.0, 1000.0)",False
9,P09099,b3564,XYLK,atp_c + xylu__D_c --> adp_c + h_c + xu5p__D_c,"(0.0, 1000.0)","(0, 0)",True


In [6]:
mutant_xylitol.genes.b0063

Gene identifier,b0063
Name,araB
Memory address,0x07f26c81a4910
Functional,True
In 3 reaction(s),"XYLK2, XYLK, RBK_L1"


mutant_xylitol.gene

In [7]:
# copy model to create mutant
mutant_isobutyric = model.copy()

# create list of genes to KO
ko_list = pd.read_csv('data/df_isobutyric_knockouts.csv', index_col=0).dropna(subset = ['id_bigg'])
ko_list.iML1515_idx = ko_list.iML1515_idx.astype(int)
#ko_list

Read LP format model from file /tmp/tmpj7a_yzll.lp
Reading time = 0.05 seconds
: 1877 rows, 5424 columns, 21150 nonzeros
Read LP format model from file /tmp/tmpz2z2tzh0.lp
Reading time = 0.04 seconds
: 1877 rows, 5424 columns, 21150 nonzeros


In [8]:
for i in ko_list.iML1515_idx:
    mutant_isobutyric.genes[i].knock_out()

isobutyric_gene_ko = evaluate_knockout(model, mutant_isobutyric, ko_list.iML1515_idx)
isobutyric_gene_ko.to_csv('result/iML1515_isobutyric_strain_ko_result.csv')   
isobutyric_gene_ko

gene: b0903 | uniprot: P09373
number of reactions involved: 2
------------------------------------------------------------
gene: b0871 | uniprot: P07003
number of reactions involved: 1
------------------------------------------------------------
gene: b3114 | uniprot: P42632
number of reactions involved: 2
------------------------------------------------------------
gene: b3952 | uniprot: P32675
number of reactions involved: 1
------------------------------------------------------------
gene: b4381 | uniprot: P0A6L0
number of reactions involved: 1
------------------------------------------------------------
gene: b1378 | uniprot: P52647
number of reactions involved: 1
------------------------------------------------------------
gene: b3011 | uniprot: Q46856
number of reactions involved: 3
------------------------------------------------------------
gene: b0063 | uniprot: P08204
number of reactions involved: 3
------------------------------------------------------------
gene: b0062 | un

,Entry,id_BIGG,reaction_id,reaction,normal_bound,mutant_bound,ko_result
0,P09373,b0903,PFL,coa_c + pyr_c --> accoa_c + for_c,"(0.0, 1000.0)","(0, 0)",True
1,P09373,b0903,OBTFL,2obut_c + coa_c --> for_c + ppcoa_c,"(0.0, 1000.0)","(0, 0)",True
2,P07003,b0871,POX,h2o_c + pyr_c + q8_c --> ac_c + co2_c + q8h2_c,"(0.0, 1000.0)","(0, 0)",True
3,P42632,b3114,PFL,coa_c + pyr_c --> accoa_c + for_c,"(0.0, 1000.0)","(0, 0)",True
4,P42632,b3114,OBTFL,2obut_c + coa_c --> for_c + ppcoa_c,"(0.0, 1000.0)","(0, 0)",True
5,P32675,b3952,PFL,coa_c + pyr_c --> accoa_c + for_c,"(0.0, 1000.0)","(0, 0)",True
6,P0A6L0,b4381,DRPA,2dr5p_c --> acald_c + g3p_c,"(0.0, 1000.0)","(0, 0)",True
7,P52647,b1378,POR5,coa_c + 2.0 flxso_c + pyr_c <=> accoa_c + co2_...,"(-1000.0, 1000.0)","(0, 0)",True
8,Q46856,b3011,ALDD2y,acald_c + h2o_c + nadp_c --> ac_c + 2.0 h_c + ...,"(0.0, 1000.0)","(0, 0)",True
9,Q46856,b3011,13PPDH2,3hppnl_c + h_c + nadph_c <=> 13ppd_c + nadp_c,"(-1000.0, 1000.0)","(0.0, 1000.0)",True


# Add Reactions
## Add Metabolites

In [9]:
model_yeast = load_model('iMM904')
model_yeast.solver = 'gurobi'
model_yeast

Name,iMM904
Memory address,0x07f26c42ef590
Number of metabolites,1226
Number of reactions,1577
Number of groups,0
Objective expression,0.0 + 1.0*BIOMASS_SC5_notrace - 1.0*BIOMASS_SC5_notrace_reverse_93090
Compartments,"cytosol, extracellular space, mitochondria, peroxisome/glyoxysome, endoplasmic reticulum, vacuole, golgi apparatus, nucleus"


In [10]:
# add metabolites
isobuta_c = Metabolite(
    'isobuta_c',
    formula='C4H8O2',
    name='2-methylpropanoic-acid',
    compartment='c')

isobuta_e = Metabolite(
    'isobuta_e',
    formula='C4H8O2',
    name='2-methylpropanoic-acid',
    compartment='e')

mppal_c = Metabolite(
    '2mppal_c',
    formula='C4H8O',
    name='2-methylpropanal',
    compartment='c')

xylt_c = model_yeast.metabolites.xylt_c
xylt_e = model_yeast.metabolites.xylt_e
xyl__D_c = model.metabolites.xyl__D_c
h_c = model.metabolites.h_c
co2_c = model.metabolites.co2_c
mob_c = model.metabolites.get_by_id('3mob_c')
h2o_c = model.metabolites.h2o_c
nad_c = model.metabolites.nad_c
nadh_c = model.metabolites.nadh_c
nadp_c = model.metabolites.nadp_c
nadph_c = model.metabolites.nadph_c

## Isobutyric Mutant
### _Reaction 1: 3-Methyl-2-oxobutanoate => 2-methylpropanal_

In [11]:
# define reaction
reaction = Reaction('kivD_heterologous')
reaction.name = 'alpha-ketoisovalerate-decarboxylase'
reaction.subsystem = 'Alternate Carbon Metabolism'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default

In [12]:
reaction.add_metabolites({
    mob_c: -1.0,
    h_c: -1.0,
    co2_c: 1.0,
    mppal_c: 1.0
})

reaction.reaction  

'3mob_c + h_c --> 2mppal_c + co2_c'

In [13]:
reaction.gene_reaction_rule = '( kivD )'
reaction.genes

frozenset({<Gene kivD at 0x7f26c486ad10>})

In [14]:
mutant_isobutyric.add_reactions([reaction])

### _Reaction 2: 2-methylpropanal =>  2-methylpropanoic-acid_

In [15]:
# define reaction
reaction = Reaction('ALDD19xr_mppal')
reaction.name = 'Aldehyde dehydrogenase (2-methylpropanal, NAD)'
reaction.subsystem = 'Alternate Carbon Metabolism'
reaction.lower_bound = -1000.  # This is the default
reaction.upper_bound = 1000.  # This is the default

In [16]:
reaction.add_metabolites({
    h2o_c: -1.0,
    nad_c: -1.0, 
    mppal_c: -1.0,
    h_c: 2.0,
    nadh_c: 1.0,
    isobuta_c: 1.0
})

reaction.reaction  

'2mppal_c + h2o_c + nad_c <=> 2.0 h_c + isobuta_c + nadh_c'

In [17]:
reaction.gene_reaction_rule = '( b1385 )'
reaction.genes

frozenset({<Gene b1385 at 0x7f26c4879c90>})

In [18]:
mutant_isobutyric.add_reactions([reaction])

### _Reaction 3: 2-methylpropanoic-acid transport_

In [19]:
# define reaction
reaction = Reaction('isobutat')
reaction.name = '2-methylpropanoic-acid transport'
reaction.subsystem = 'Transport, Extracellular' #TRANSPORT
reaction.lower_bound = -1000.  # This is the default
reaction.upper_bound = 1000.  # This is the default

In [20]:
reaction.add_metabolites({
    isobuta_c: -1.0,
    isobuta_e: 1.0
})

reaction.reaction  

'isobuta_c <=> isobuta_e'

In [21]:
reaction.gene_reaction_rule = '( isobuta_transporter )'
reaction.genes

frozenset({<Gene isobuta_transporter at 0x7f26c3dd82d0>})

In [22]:
mutant_isobutyric.add_reactions([reaction])

### _Reaction 4: 2-methylpropanoic-acid => exchange_

In [23]:
# define reaction
reaction = Reaction('EX_isobuta_e')
reaction.name = '2-methylpropanoic-acid exchange'
reaction.subsystem = 'Extracellular exchange'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default

In [24]:
reaction.add_metabolites({
    isobuta_e: -1.0
})

reaction.reaction  

'isobuta_e --> '

In [25]:
mutant_isobutyric.add_reactions([reaction])

### _Reaction 4: Acetate Transporter_
#For some reason, we did not find acetate transporter in iML1515 http://bigg.ucsd.edu/universal/reactions/ACtr

In [26]:
#reaction = model_yeast.reactions.ACtr
#mutant_isobutyric.add_reactions([reaction])
#mutant_isobutyric.reactions.ACtr.bounds = (-1000.0, 1000.0)

### Write Model

In [27]:
cobra.io.write_sbml_model(mutant_isobutyric, "result/iML1515_isobutyric_strain.xml")
cobra.io.save_json_model(mutant_isobutyric, "result/iML1515_isobutyric_strain.json")

## Xylitol Mutant
### _Reaction 1: NADPH-dependent D-xylose reductase_

In [28]:
reaction = model_yeast.reactions.XYLR

In [29]:
mutant_xylitol.add_reactions([reaction])
mutant_xylitol.reactions.XYLR.bounds = (0.0, 1000.0)

### _Reaction 2: Xylitol transport via passive diffusion_

In [30]:
reaction = model_yeast.reactions.XYLTt
reaction.reaction

'xylt_e <=> xylt_c'

In [31]:
mutant_xylitol.add_reactions([reaction])
mutant_xylitol.reactions.XYLTt.bounds = (-1000.0, 1000.0)

### _Reaction 3: Xylitol Exchange_

In [32]:
# define reaction
reaction = Reaction('EX_xylt_e')
reaction.name = 'Xylitol Exchange'
reaction.subsystem = 'Extracellular exchange'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default

In [33]:
reaction.add_metabolites({
    xylt_e: -1.0
})

reaction.reaction 

'xylt_e --> '

In [34]:
mutant_xylitol.add_reactions([reaction])

### Write Model

In [35]:
cobra.io.write_sbml_model(mutant_xylitol, "result/iML1515_xylitol_strain.xml")
cobra.io.save_json_model(mutant_xylitol, "result/iML1515_xylitol_strain.json")